In [ ]:
import os, sys, import_ipynb, ipynb
if os.getcwd() == 'C:\\Users\\admin\\Desktop\\retail_data_analysis\\code':
    print ("Already in code directory")
else:
    os.chdir('..//code')
        
# import python libraries
from importLibraries import *


def fun_pricing():
    
    # Calculate optimal price for total revenue maximization 
    os.chdir('..//input')
    input_data_preProcessed = pd.read_csv("input_data_preProcessed.csv")
    
    # convert created_at to datetime object
    input_data_preProcessed['new_Only_InvoiceDate'] = pd.to_datetime(input_data_preProcessed['new_Only_InvoiceDate'], format='%Y-%m-%d %H:%M:%S.%f')
    input_data_preProcessed['new_Only_InvoiceDate'] = input_data_preProcessed['new_Only_InvoiceDate'].dt.date
       
    tmpdf = input_data_preProcessed.groupby(['new_Only_InvoiceDate'])['Quantity'].agg('sum').reset_index().rename(columns={'Quantity':'SumQuantity'})
        
    idx = pd.date_range('2010-12-01', '2011-02-28')
    tmpdf = pd.Series(tmpdf['SumQuantity'].values, index=tmpdf['new_Only_InvoiceDate'])
    tmpdf = tmpdf.reindex(idx)
    tmpdf = tmpdf.interpolate(method ='linear', limit_direction ='both')    
    display(tmpdf.head())
    
    # Plot
    fig, ax1 = plt.subplots(figsize = (10,10))    
    tmpdf.plot()
    plt.show
    plt.savefig('new_Only_InvoiceDate_vs_SumQuantity.jpg')
    
    
    # Split data into training and testing
    split_point = len(tmpdf) - 10
    train, test = tmpdf[0:split_point], tmpdf[split_point:]
    display('train %d, test %d' % (len(train), len(test)))
    
    # If we see that the ACF is greater than zero till 10 lags (x axis) then we can say that there is a positive correlation with the first 10 lags. That is significant for the first 5 lags if till first 5 lags the curve is greater than upper conf interval line. It means that the AR may be 5.
    autocorrelation_plot(train)
    #plt.show
    
    # fit model
    # The distribution of the residual errors is displayed in the density plot. The density plot suggest the errors are Gaussian, but may not be centered on zero
    # If there is non zero mean in residual then it indicates bias in the prediction
    model = ARIMA(train, order=(5,1,0))
    model_fit = model.fit(disp=0)
    display(model_fit.summary())
    # plot residual errors
    residuals = DataFrame(model_fit.resid)
    residuals.plot()
    plt.show()
    residuals.plot(kind='kde')
    plt.show()
    display(residuals.describe())
    
    # Build the model. set d=2, iteratively increase p to up to 5 and q up to 5 and see  which model is giving minimum AIC. Also check the actuals and forecasts in chart. The P-values of the X terms should ideally be less then < 0.05
    model = ARIMA(train, order=(3, 2, 1))  
    fitted = model.fit(disp=-1)  
    display(fitted.summary())

    # Forecasting
    fc, se, conf = fitted.forecast(10, alpha=0.05)  # 95% conf

    # Making it a pandas series
    fc_series = pd.Series(fc, index=test.index)
    lower_series = pd.Series(conf[:, 0], index=test.index)
    upper_series = pd.Series(conf[:, 1], index=test.index)
    display(fc_series)

    # Plot
    plt.figure(figsize=(12,5), dpi=100)
    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(fc_series, label='forecast')
    plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8)
    plt.show()
    
    
    